Examine the odd case that the value for the heritability estimate can be higher than the upper error bar.

In [ ]:
source("/scratch/PI/mrivas/users/erflynn/sex_div_gwas/mixture_model_scripts/heritability_utils.R")
 source("/scratch/PI/mrivas/users/erflynn/sex_div_gwas/mixture_model_scripts/model_utils.R")
    DATA.FOLDER <- "/scratch/PI/mrivas/users/erflynn/sex_div_gwas/data/1009"


In [ ]:
trait <- "arm_fp"

  load(sprintf("%s/f_%s.RData", DATA.FOLDER, trait))
  load(sprintf("%s/dat_%s.RData", DATA.FOLDER, trait))

        # extract all estimate
        list_of_draws <- rstan::extract(fit1)
        pi.draws <- list_of_draws$pi
        p <- pi.draws
        s.draws <- list_of_draws$Sigma
        Sigma <- s.draws

        # extract lower + upper pi
        ordered.p <- p[order(p[,2]),] # ordering p by the non-null component 
        p.lower <- ordered.p[0.025*nrow(ordered.p),]
        p.upper <- ordered.p[0.975*nrow(ordered.p),]
        p.center <- ordered.p[0.50*nrow(ordered.p),]
    
        # extract lower + upper sigma
        ordered.S <- Sigma[order(Sigma[,1,1]),,]
        s.upper <- ordered.S[0.975*dim(Sigma)[1],,]
        s.lower <- ordered.S[0.025*dim(Sigma)[1],,]
        s.center <- ordered.S[0.50*dim(Sigma)[1],,]

In [ ]:
# pi and Sigma estimates - also match what is written by the summary
p.lower
p.center
p.upper

s.lower
s.center
s.upper

print(fit1, pars=c("Sigma", "pi"), probs=c(0.025, 0.5, 0.975), digits_summary=7)

In [ ]:

dat2 <- dat
# assign snps to components 
        dat2$categories <- NULL
        dat.u <- labelCategories(dat2, s.upper, p.upper) # implemented in heritability_utils
        dat.l <- labelCategories(dat2, s.lower, p.lower)
        dat.c <- labelCategories(dat2, s.center, p.center)


In [ ]:
# number per category
table(dat.l$categories)
table(dat.c$categories)
table(dat.u$categories)

In [ ]:
# ok estimate heritability (this function is implemented in heritability_utils)     
h.up <- overallHeritability(dat.u, s.upper, p.upper)
        h.low <- overallHeritability(dat.l, s.lower, p.lower)
        h.center <- overallHeritability(dat.c, s.center, p.center)


In [ ]:
h.up
h.center
h.low

In [ ]:
# look at the calc for upper (this is inside overall_heritability)
se.p2 <- dat.u$dat$SE[dat.u$categories==2,]
n <- nrow(se.p2)
num_i <- n*(p.upper[2])*s.upper[1,1]
    
h_i <- num_i/(num_i + sum(se.p2[,1]))
sum(se.p2[,1])
num_i
h_i
num_i <- n*(p.upper[2])*s.upper[2,2]
    
h_2 <- num_i/(num_i + sum(se.p2[,2]))
sum(se.p2[,2])
num_i
h_2

In [ ]:
# look at the calc for center (this is inside overall_heritability)
se.p2 <- dat.u$dat$SE[dat.c$categories==2,]
n <- nrow(se.p2)
num_i <- n*(p.center[2])*s.center[1,1]
    
h_i <- num_i/(num_i + sum(se.p2[,1]))
sum(se.p2[,1])
num_i
h_i
num_i <- n*(p.center[2])*s.center[2,2]
    
h_2 <- num_i/(num_i + sum(se.p2[,2]))
sum(se.p2[,2])
num_i
h_2

In [ ]:

# how do we see this swap
0.09/(0.26+0.09) # higher input --> lower output
0.064/(0.182+0.064) # lower input --> higher output